# Transformers

In [18]:
from transformers import BertTokenizer, BertModel

#tokenizer = BertTokenizer.from_pretrained("OpenAIGPTLMHeadModel")
model = BertModel.from_pretrained("wietsedv/bert-base-dutch-cased")

In [13]:
https://huggingface.co/wietsedv/bert-base-dutch-cased?text=gaan+we+%5BMASK%5D+

https://huggingface.co/blog/how-to-generate

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased")
model = AutoModelWithLMHead.from_pretrained("wietsedv/bert-base-dutch-cased")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Some weights of the model checkpoint at wietsedv/bert-base-dutch-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at wietsedv/bert-base-dutch-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to us

In [2]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 13 s, sys: 848 ms, total: 13.8 s
Wall time: 13.9 s


In [54]:
sample_texts = tweede_kamer.query("party == 'VVD'").sample(1).text.values.tolist()

In [55]:
inputs = " ".join(sample_texts)

In [56]:
inputs

'De argumenten zijn al gegeven in de schriftelijke inbreng; daar staan zij aangegeven. Wij hebben allen, zo mag ik aannemen, veel reacties gekregen en gesprekken gevoerd met vakbeweging, met werkgevers, met organisaties op dit terrein. Wij kennen de argumenten. U vraagt almaar door naar de bekende weg. Nogmaals: ik ga het niet uitlokken, ik wacht niet tot het gebeurd is. Dan zitten wij namelijk hier bij elkaar en zeggen wij: tjonge, wat erg dat dit toch gebeurd is.'

In [57]:
input_ids = tokenizer.encode(inputs, return_tensors='pt')

In [58]:
input_ids.shape

torch.Size([1, 99])

In [59]:
N = input_ids.shape[1]

In [66]:
greedy_output = model.generate(input_ids, max_length= N +20)

In [67]:
tt = len(inputs)

In [68]:
print(inputs)
print("--------------")
out = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
zz = len(out)
out[tt:zz]


De argumenten zijn al gegeven in de schriftelijke inbreng; daar staan zij aangegeven. Wij hebben allen, zo mag ik aannemen, veel reacties gekregen en gesprekken gevoerd met vakbeweging, met werkgevers, met organisaties op dit terrein. Wij kennen de argumenten. U vraagt almaar door naar de bekende weg. Nogmaals: ik ga het niet uitlokken, ik wacht niet tot het gebeurd is. Dan zitten wij namelijk hier bij elkaar en zeggen wij: tjonge, wat erg dat dit toch gebeurd is.
--------------


'dat is niet zo erg, hoor. Nogmaals : wij vragen het niet meer meer terug'